In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}

In [2]:
import qualified Data.Vector as V
import Data.Vector ((//), (!))
import Data.List (unfoldr)
import qualified Data.Map.Strict as M

In [3]:
type Memory = V.Vector Int
type Redist = (Int, Int, Memory)
type History = M.Map Memory Int

In [4]:
vec0 = V.fromList [0, 2, 7, 0]
vec0

[0,2,7,0]

In [5]:
redistStart :: Memory -> Redist
redistStart vec0 = (current, toDistribute, startVec)
    where origin = V.maxIndex vec0
          toDistribute = vec0!origin
          current = (origin + 1) `mod` (length vec0)
          startVec = vec0 // [(origin, 0)]

In [6]:
redistStart $ V.fromList [0, 2, 7, 0]

(3,7,[0,2,0,0])

In [7]:
redistR :: Redist -> Memory
redistR (_, 0, vec) = vec
redistR (i, n, vec) = redistR (i', n', vec')
    where n' = n - 1
          i' = (i + 1) `mod` (length vec)
          vec' = vec // [(i, vec!i + 1)]

In [8]:
redist = redistR . redistStart

In [9]:
redist $ V.fromList [0, 2, 7, 0]

[2,4,1,2]

In [10]:
redist $ redist $  V.fromList [0, 2, 7, 0]

[3,1,2,3]

In [11]:
redistSeq = unfoldr redistU
    where redistU vec = Just (redist vec, redist vec)

In [12]:
take 8 $ zip [0..] $ redistSeq $ V.fromList [0, 2, 7, 0]

[(0,[2,4,1,2]),(1,[3,1,2,3]),(2,[0,2,3,4]),(3,[1,3,4,1]),(4,[2,4,1,2]),(5,[3,1,2,3]),(6,[0,2,3,4]),(7,[1,3,4,1])]

In [13]:
findRepeat :: History -> [(Int, Memory)] -> Int
findRepeat h ((n, x) : nxs) = if x `M.member` h 
                              then n + 1
                              else findRepeat (M.insert x n h) nxs
    where n0 = (M.!) h x

In [26]:
findRepeatB :: History -> [(Int, Memory)] -> Int
findRepeatB h ((n, x) : nxs) = if x `M.member` h 
                              then n - n0
                              else findRepeatB (M.insert x n h) nxs
    where n0 = (M.!) h x

In [27]:
findRepeat M.empty $ zip [0..] $ redistSeq $ V.fromList [0, 2, 7, 0]

5

In [28]:
part1 :: [Int] -> Int
-- part1 memlist = findRepeat M.empty $ zip [0..] $ redistSeq $ V.fromList memlist
part1 = (findRepeat M.empty) . (zip [0..]) . redistSeq . V.fromList

In [29]:
part2 :: [Int] -> Int
-- part1 memlist = findRepeat M.empty $ zip [0..] $ redistSeq $ V.fromList memlist
part2 = (findRepeatB M.empty) . (zip [0..]) . redistSeq . V.fromList

In [30]:
main :: IO ()
main = do 
        text <- readFile "../../data/advent06.txt"
--         text0 <- readFile "../../data/advent06.txt"
--         let text = "0 2 7 0"
        let memory = map read $ words text
        print $ part1 memory
        print $ part2 memory

In [31]:
main

7864
1695